In [ ]:


import spacy
import csv
import datetime

nlp = spacy.load("en_core_web_sm")


class ToDoList:
    def __init__(self):
        self.tasks = []

    def add_task(self, task):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.tasks.append((task, timestamp))
        print("Task added successfully!")
        self.save_tasks_to_csv("tasks.csv")  # Automatically save tasks to CSV after adding

    def view_tasks(self):
        if self.tasks:
            print("Tasks:")
            for i, (task, timestamp) in enumerate(self.tasks, 1):
                print(f"{i}. {task} - Added on: {timestamp}")
        else:
            print("No tasks.")

    def mark_task_completed(self, task_index):
        if 1 <= task_index <= len(self.tasks):
            self.tasks[task_index - 1] = (self.tasks[task_index - 1][0] + " - Completed", self.tasks[task_index - 1][1])
            print("Task marked as completed!")
            self.save_tasks_to_csv("tasks.csv")  # Automatically save tasks to CSV after marking completed
        else:
            print("Invalid task index.")

    def delete_task(self, task_index):
        if 1 <= task_index <= len(self.tasks):
            del self.tasks[task_index - 1]
            print("Task deleted successfully!")
            self.save_tasks_to_csv("tasks.csv")  # Automatically save tasks to CSV after deletion
        else:
            print("Invalid task index.")

    def save_tasks_to_csv(self, filename):
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Task', 'Timestamp'])
            writer.writerows(self.tasks)

    def load_tasks_from_csv(self, filename):
        try:
            with open(filename, 'r') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  # Skip header row
                self.tasks = [(row[0], row[1]) for row in reader]
        except FileNotFoundError:
            print("No existing tasks found. Creating a new CSV file.")
            # Create a new CSV file with the header row
            self.save_tasks_to_csv(filename)

    def interpret_command(self, command):
        doc = nlp(command)
        verb = None
        task = None

        for token in doc:
            if token.pos_ == "VERB":
                verb = token.text
            elif token.pos_ == "NOUN":
                task = token.text

        return verb, task


def main():
    todo_list = ToDoList()

    try:
        todo_list.load_tasks_from_csv("tasks.csv")
    except FileNotFoundError:
        print("No existing tasks found.")

    while True:
        user_input = input("\nEnter your command: ")
        verb, task = todo_list.interpret_command(user_input)

        if verb == "add":
            if task:
                todo_list.add_task(task)
            else:
                print("Invalid command. Please specify a task.")
        elif verb == "view":
            todo_list.view_tasks()
        elif verb == "mark":
            if task:
                todo_list.mark_task_completed(int(task))
            else:
                print("Invalid command. Please specify a task index.")
        elif verb == "delete":
            if task:
                todo_list.delete_task(int(task))
            else:
                print("Invalid command. Please specify a task index.")
        elif verb == "exit":
            print("Exiting...")
            break
        else:
            print("Invalid command. Please try again.")


if __name__ == "__main__":
    main()
